$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [1]:
import os, sys

def add_parent_lib_path(name="Leptoquarks_Searches_2023"):
    sys.path.append(sys.path[0].split(name)[0])
    
add_parent_lib_path()

In [2]:
Path_Leptoquarks_searches = os.path.dirname(os.path.dirname(os.path.realpath('XBG_Discriminator_Histograms.ipynb')))

In [3]:
from Uniandes_Framework.ml_tools import tools
from Uniandes_Framework.delphes_reader import root_analysis

import pandas as pd

from ROOT import TCanvas #Permite poner los histogramas
from ROOT import THStack #Permite graficar varios histogramas al mismo tiempo
from ROOT import TLegend #Permite poner legends cuando se sobrelapan histogramas
from ROOT import TLatex #Permite poner avisos en Latex en las graficas

Welcome to JupyROOT 6.22/06


In [4]:
channels = ["hadronic_dLQ", "hadronic_sLQ", "hadronic_non-resonant", "semileptonic_dLQ", "semileptonic_sLQ", "semileptonic_non-resonant"]
signals = ['LQ_LQ', 'Tau_LQ', 'Tau_Tau'] + ['LQ_LQ_wo_RHC', 'Tau_LQ_wo_RHC', 'Tau_Tau_wo_RHC']
Masses = [
    "1000" ,
    '1250', 
    '1500', 
    '1750', 
    '2000', 
    '2250', 
    '2500'
]
bkgs = ['ttbar', 'z_jets', 'w_jets', 'stop','ww', 'wz', 'zz']

In [5]:
try: os.mkdir('Histograms')
except: pass

def escape_braces(strings):
    escaped_strings = []
    for s in strings:
        escaped_s = s.replace('{', '{{').replace('}', '}}')
        escaped_strings.append("{}".format(escaped_s))
    return escaped_strings


for channel in channels:
    try: os.mkdir(f'Histograms/{channel}')
    except: pass    

    bkg_dict = {}
    for bkg in bkgs: bkg_dict[bkg] = os.path.join(os.sep,"disco4","pheno_csv_files","Leptoquarks_Searches",bkg,f"{bkg}_{channel}.csv")
    
    most_important_features = []
    with open(os.path.join(os.getcwd(), 'XGB_models', channel, f'Most_Important_Features.txt'), "r") as f:
        for line in f: most_important_features.append(line.strip())
    f.close()
    print(most_important_features)
    #most_important_features = escape_braces(most_important_features)                      
    for Mass in Masses:
        try: os.mkdir(f'Histograms/{channel}/M{Mass}')
        except: pass

        signal_dict = {}
        for signal in signals: 
            signal_dict[f"{signal}_{Mass}"] = os.path.join(os.sep,"disco4","pheno_csv_files","Leptoquarks_Searches",f"{signal}_{Mass}",f"{signal}_{Mass}_{channel}.csv")

        path_model_mass =  os.path.join(os.getcwd(), 'XGB_models', channel, f'M{Mass}_XGB.joblib')
        bkg_and_signal_dict = bkg_dict | signal_dict
        path_folder_mass = os.path.join(os.getcwd(), 'Histograms', channel, f'M{Mass}')
        
        with root_analysis.Quiet(): tools.hist_discriminator(path_model= path_model_mass, csv_dict = bkg_and_signal_dict, path_to_save= path_folder_mass, best_features= most_important_features)

['pT_{#tau_{1}}(GeV)', '#Delta R_{#tau_{1}b_{2}}', '#eta_{b_{2}}', '#Delta #phi_{#tau_{1}b_{1}}', '#Delta R_{#tau_{1}b_{1}}', '#Delta #eta_{#tau_{1}#tau_{2}}', '#Delta #phi_{b_{1}MET}', '#Delta #phi_{b_{2}MET}', '#phi_{#tau_{1}}', '#phi_{#tau_{2}}', 'sT(GeV)']
['sT(GeV)', 'Q_{#tau_{1}}Q_{#tau_{2}}', '#Delta #phi_{b_{1}MET}', '#Delta R_{#tau_{1}b_{1}}', '#Delta #phi_{#tau_{2}MET}', '#Delta #eta_{#tau_{1}#tau_{2}}', '#Delta R_{#tau_{2}b_{1}}', '#phi_{#tau_{1}}', '#Delta #phi_{#tau_{1}b_{1}}', '#phi_{#tau_{2}}']
['sT(GeV)', '#Delta R_{#tau_{1}#tau_{2}}', 'Q_{#tau_{1}}Q_{#tau_{2}}', '#Delta #phi_{#tau_{2}MET}', '#Delta #phi_{#tau_{1}#tau_{2}}', '#Delta pT_{#tau_{2}MET}(GeV)', '#phi_{#tau_{1}}', '#eta_{#tau_{1}}', '#phi_{MET}', '#phi_{#tau_{2}}']
['sT(GeV)', 'Q_{lep_{1}}Q_{#tau_{1}}', '#Delta pT_{b_{2}MET}(GeV)', '#Delta #phi_{lep_{1}#tau_{1}}', '#Delta R_{#tau_{1}b_{1}}', '#Delta #eta_{b_{1}b_{2}}', '#Delta #phi_{lep_{1}MET}', '#Delta #eta_{lep_{1}#tau_{1}}', '#phi_{MET}', '#phi_{b_{2}}']
